# 실습 세션 1: LLM 및 프롬프트 엔지니어링 활용

본 세션에서는 Azure AI Foundry에서 배포한 LLM을 활용하여 Chat Playground와 Jupyter Notebook에서 다양한 프롬프트 엔지니어링 기법을 실습합니다.

## 1. 패키지 설치 및 Azure OpenAI 연결 설정

실습에 필요한 패키지를 설치하고, Azure OpenAI 서비스에 연결하기 위한 환경설정을 진행합니다.

In [1]:
# 필요한 라이브러리 설치
!pip install -r requirements.txt

In [2]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

# .env 파일에서 환경 변수 로드 (없는 경우 직접 입력)
load_dotenv()

# Azure OpenAI 설정
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# AzureOpenAI 클라이언트 생성
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version=api_version
)

# 모델 배포 이름
# gpt-4o mini 배포 시 아래와 같이 이름을 지정할 수 있습니다.
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME") # 배포된 모델 이름

## 2. Azure OpenAI LLM 모델 연결 및 응답 테스트

이 단계에서는 앞서 배포한 gpt-4o mini 등 Azure OpenAI LLM 모델이 정상적으로 연결되어 있는지 확인하고, 실제로 간단한 메시지를 보내 응답을 받아봅니다.

- 환경 변수(.env)에 입력한 정보로 Azure OpenAI 서비스에 연결합니다.
- 배포된 모델 이름(deployment_name)을 사용해 API 요청을 보냅니다.
- 연결 및 응답 테스트를 통해 모델이 정상적으로 동작하는지 확인할 수 있습니다.
- 만약 오류가 발생하면, 환경 변수 값과 Azure Portal의 배포 상태를 다시 확인하세요.

In [3]:
# 배포된 모델 목록 확인
try:
    # 참고: 현재 버전의 AzureOpenAI 클라이언트에서는 
    # 모든 배포 목록을 직접 확인하는 기능이 제한적일 수 있습니다.
    # 아래 코드는 예시일 뿐이며, 실제 환경에 따라 다를 수 있습니다.
    print("배포된 모델 정보:")
    print(f"- 현재 사용 중인 모델: {deployment_name}")
    print("Azure Portal에서 배포된 모든 모델을 확인할 수 있습니다.")

    # 배포된 모델 테스트 (간단한 테스트)
    test_response = client.chat.completions.create(
        model=deployment_name,
        messages=[{"role": "user", "content": "안녕하세요"}],
        max_tokens=10
    )
    print(f"\n모델 응답 테스트 성공: {test_response.choices[0].message.content}")
except Exception as e:
    print(f"모델 확인 중 오류 발생: {e}")
    print("Azure Portal에서 직접 배포 상태를 확인해주세요.")

배포된 모델 정보:
- 현재 사용 중인 모델: gpt-4o-mini
Azure Portal에서 배포된 모든 모델을 확인할 수 있습니다.

모델 응답 테스트 성공: 안녕하세요! 어떻게 도와드릴까요?


## 3. 기본 프롬프트 엔지니어링

이 단계에서는 LLM의 응답 품질을 높이기 위한 다양한 프롬프트 엔지니어링 기법을 실습합니다.

- 명확한 지시사항, 역할 부여, 단계별 사고 유도, Few-Shot 학습 등 다양한 프롬프트 작성법을 실습합니다.
- 각 기법을 적용한 프롬프트와 적용하지 않은 프롬프트의 차이를 비교해볼 수 있습니다.
- 실습을 통해 실제 업무나 프로젝트에서 활용할 수 있는 프롬프트 설계 역량을 키울 수 있습니다.
- 아래 예시 코드를 직접 수정·실행하며 다양한 결과를 실험해보세요.

In [4]:

def get_completion(prompt, model=deployment_name):
    """지정된 프롬프트와 모델을 사용하여 Azure OpenAI API에 요청을 보내고 응답을 반환합니다."""
    # 메시지 구성
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    # 채팅 완성 요청
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=4096
    )
    
    # 응답에서 메시지 내용 추출
    return response.choices[0].message.content

# 간단한 프롬프트 테스트
simple_prompt = "Azure란 무엇인가요?"
print(get_completion(simple_prompt))

Azure는 마이크로소프트(Microsoft)에서 제공하는 클라우드 컴퓨팅 플랫폼이자 서비스입니다. Azure는 다양한 서비스와 솔루션을 제공하여 기업과 개발자가 애플리케이션을 구축, 배포 및 관리할 수 있도록 지원합니다. 주요 기능과 특징은 다음과 같습니다:

1. **컴퓨팅 서비스**: 가상 머신, 컨테이너 서비스, 서버리스 컴퓨팅 등 다양한 컴퓨팅 옵션을 제공합니다.

2. **스토리지**: Blob 스토리지, 파일 스토리지, 데이터베이스 서비스 등 다양한 데이터 저장 옵션을 지원합니다.

3. **데이터베이스**: Azure SQL Database, Cosmos DB, Azure Database for PostgreSQL 등 다양한 데이터베이스 서비스를 제공합니다.

4. **네트워킹**: 가상 네트워크, VPN, 로드 밸런서, CDN(Content Delivery Network) 등 네트워크 관련 서비스도 포함되어 있습니다.

5. **AI 및 머신러닝**: Azure Machine Learning, Cognitive Services 등 AI 및 머신러닝 관련 도구와 서비스도 제공합니다.

6. **보안 및 관리**: Azure Security Center, Azure Active Directory 등 보안 및 관리 기능을 통해 데이터와 애플리케이션을 보호하고 관리할 수 있습니다.

7. **IoT**: Azure IoT Hub와 같은 서비스를 통해 IoT 솔루션을 구축하고 관리할 수 있습니다.

Azure는 사용자가 필요에 따라 리소스를 쉽게 확장하거나 축소할 수 있는 유연성을 제공하며, 전 세계 여러 지역에 데이터 센터를 운영하여 높은 가용성과 신뢰성을 보장합니다.


### 3.1 명확한 지시사항 제공하기

LLM에게 명확한 지시사항을 제공하면 더 좋은 결과를 얻을 수 있습니다.

In [5]:
# 명확하지 않은 지시사항
unclear_prompt = "클라우드 컴퓨팅에 대해 알려줘."
print("\n=== 명확하지 않은 지시사항 ===\n")
print(get_completion(unclear_prompt))

# 명확한 지시사항
clear_prompt = """클라우드 컴퓨팅에 대해 다음 형식으로 설명해줘:
1. 정의 (50단어 이내)
2. 주요 이점 (5가지 나열)
3. 주요 서비스 제공업체 (3개)
4. Azure의 주요 서비스 (5가지)
"""
print("\n=== 명확한 지시사항 ===\n")
print(get_completion(clear_prompt))


=== 명확하지 않은 지시사항 ===

클라우드 컴퓨팅(Cloud Computing)은 인터넷을 통해 서버, 저장소, 데이터베이스, 네트워크, 소프트웨어 및 분석과 같은 컴퓨터 자원을 제공하는 기술입니다. 이를 통해 사용자는 물리적인 하드웨어나 소프트웨어를 소유하지 않고도 IT 자원을 쉽게 사용할 수 있습니다. 클라우드 컴퓨팅의 주요 특징과 장점은 다음과 같습니다.

### 주요 특징

1. **온디맨드 셀프서비스(On-demand self-service)**: 사용자는 필요할 때 언제든지 자원을 요청하고 사용할 수 있습니다.

2. **광범위한 네트워크 접근성(Broad network access)**: 인터넷만 있으면 다양한 디바이스(컴퓨터, 모바일 기기 등)에서 클라우드 서비스에 접근할 수 있습니다.

3. **자원 풀링(Resource pooling)**: 클라우드 서비스 제공자는 여러 고객의 자원을 통합하여 효율적으로 관리하고 배분합니다.

4. **빠른 확장성(Rapid elasticity)**: 필요에 따라 자원을 신속하게 확장하거나 축소할 수 있는 기능입니다.

5. **측정 가능한 서비스(Measured service)**: 사용자는 실제 사용량에 따라 비용을 지불하며, 클라우드 제공자는 사용량을 모니터링하고 관리합니다.

### 클라우드 컴퓨팅의 유형

1. **공공 클라우드(Public Cloud)**: 여러 사용자가 공유하는 클라우드 인프라로, 일반 대중에게 서비스가 제공됩니다. 예: AWS, Microsoft Azure, Google Cloud.

2. **사설 클라우드(Private Cloud)**: 특정 조직 전용으로 구축된 클라우드 인프라로, 보안과 제어가 더 강화됩니다.

3. **하이브리드 클라우드(Hybrid Cloud)**: 공공 클라우드와 사설 클라우드를 결합하여 각 환경의 장점을 활용하는 모델입니다.

4. **멀티 클라우드(Multi-Cloud)**: 여러 공공 클라우드 서비스를 조합하여 사용하는 전략으로, 특정 공급

### 3.2. 역할 부여하기 (Role Prompting)

LLM에게 특정 역할을 부여하면 해당 관점에서 응답을 생성할 수 있습니다.

In [6]:
# 역할 없는 프롬프트
no_role_prompt = "Azure Kubernetes Service(AKS)의 장점을 설명해줘."
print("\n=== 역할 없는 프롬프트 ===\n")
print(get_completion(no_role_prompt))

# 역할 부여 프롬프트
role_prompt = """당신은 5년 경력의 클라우드 아키텍트입니다. 
기업의 IT 관리자에게 Azure Kubernetes Service(AKS)를 도입해야 하는 이유와 장점을 
기술적 측면과 비즈니스 측면에서 설득력 있게 설명해주세요."""
print("\n=== 역할 부여 프롬프트 ===\n")
print(get_completion(role_prompt))


=== 역할 없는 프롬프트 ===

Azure Kubernetes Service (AKS)는 Microsoft Azure에서 제공하는 관리형 Kubernetes 서비스로, 여러 가지 장점을 가지고 있습니다. 다음은 AKS의 주요 장점입니다:

1. **관리 용이성**: AKS는 Kubernetes 클러스터의 배포, 관리 및 운영을 단순화합니다. Azure가 클러스터의 제어 평면을 관리하므로 사용자는 애플리케이션에 집중할 수 있습니다.

2. **자동화된 업데이트**: AKS는 Kubernetes 버전 업그레이드 및 패치 관리를 자동화하여 최신 기능과 보안 업데이트를 쉽게 적용할 수 있습니다.

3. **스케일링**: AKS는 수요에 따라 클러스터를 수평적으로 자동으로 확장하거나 축소할 수 있는 기능을 제공합니다. 이를 통해 비용 효율적으로 리소스를 관리할 수 있습니다.

4. **통합된 Azure 서비스**: AKS는 Azure의 다른 서비스와 원활하게 통합됩니다. 예를 들어, Azure Monitor, Azure Active Directory, Azure DevOps 등과의 통합을 통해 모니터링, 보안 및 CI/CD 파이프라인을 쉽게 설정할 수 있습니다.

5. **보안**: Azure는 엔터프라이즈급 보안 기능을 제공합니다. AKS는 Azure Active Directory 통합, RBAC(역할 기반 접근 제어), 네트워크 보안 그룹 등 다양한 보안 기능을 통해 클러스터를 안전하게 보호합니다.

6. **비용 효율성**: AKS는 제어 평면에 대한 비용이 무료이며, 사용자는 노드와 관련된 리소스에 대해서만 비용을 지불합니다. 이를 통해 필요에 따라 비용을 최적화할 수 있습니다.

7. **다양한 배포 옵션**: AKS는 온프레미스, 클라우드 및 하이브리드 환경에서 배포할 수 있는 유연성을 제공합니다. 사용자는 필요에 따라 다양한 환경에서 애플리케이션을 운영할 수 있습니다.

8. **커뮤니티와 생태계**: Kubernetes는 오픈 소스 생태계의 일부로,

### 3.3. 단계별 사고 유도 (Chain-of-Thought)

복잡한 문제를 해결할 때는 LLM에게 단계별로 생각하도록 유도하는 것이 효과적입니다.

In [7]:
# 직접적인 질문
direct_prompt = "Azure에서 3-tier 웹 애플리케이션을 구축하는 방법을 알려줘."
print("\n=== 직접적인 질문 ===\n")
print(get_completion(direct_prompt))

# 단계별 사고 유도 프롬프트
cot_prompt = """Azure에서 3-tier 웹 애플리케이션을 구축하려고 합니다. 
다음 단계에 따라 설계 방법을 설명해주세요:

1. 먼저, 각 계층(프레젠테이션, 비즈니스 로직, 데이터)에 적합한 Azure 서비스를 선택하세요.
2. 다음으로, 이러한 서비스들이 어떻게 연결되는지 설명하세요.
3. 그 다음, 보안 측면에서 고려해야 할 사항을 설명하세요.
4. 마지막으로, 확장성을 위한 설계 방법을 제안하세요.

각 단계별로 구분하여 자세히 설명해주세요."""
print("\n=== 단계별 사고 유도 프롬프트 ===\n")
print(get_completion(cot_prompt))


=== 직접적인 질문 ===

Azure에서 3-tier 웹 애플리케이션을 구축하는 과정은 여러 단계로 나누어질 수 있습니다. 3-tier 아키텍처는 일반적으로 프레젠테이션 레이어 (UI), 애플리케이션 레이어 (로직), 데이터 레이어 (데이터베이스)로 구성됩니다. 아래는 Azure에서 3-tier 웹 애플리케이션을 구축하는 기본적인 단계입니다.

### 1. 요구 사항 정의
- 애플리케이션의 기능, 성능, 보안 요구 사항을 정의합니다.

### 2. Azure 리소스 그룹 생성
- Azure 포털에 로그인하고 새로운 리소스 그룹을 생성합니다. 리소스 그룹은 관련된 Azure 리소스를 그룹화하는 데 사용됩니다.

### 3. 프레젠테이션 레이어 구축
- **Azure App Service**: 웹 애플리케이션의 프론트엔드 UI를 호스팅하기 위해 Azure App Service를 사용합니다. ASP.NET, Node.js, Java 등 다양한 프레임워크를 사용할 수 있습니다.
  - 새로운 App Service를 생성하고 애플리케이션 코드를 배포합니다.

### 4. 애플리케이션 레이어 구축
- **Azure App Service 또는 Azure Functions**: 비즈니스 로직을 처리하기 위해 Azure App Service를 추가로 사용하거나, 서버리스 아키텍처를 원한다면 Azure Functions를 사용할 수 있습니다.
  - 필요한 API를 개발하고 Azure App Service에 배포합니다.

### 5. 데이터 레이어 구축
- **Azure SQL Database**: 관계형 데이터베이스가 필요한 경우 Azure SQL Database를 사용합니다. NoSQL 데이터베이스가 필요한 경우 Azure Cosmos DB와 같은 서비스를 사용할 수 있습니다.
  - 데이터베이스를 생성하고 테이블 및 스키마를 정의합니다.

### 6. 네트워킹 구성
- **Azure Virtual Network**: 필요한 경우, 보안을 위해 Azure Virtual

### 3.4. Few-Shot 학습

몇 가지 예시를 제공하여 LLM이 패턴을 학습하도록 유도할 수 있습니다.

In [8]:
# Few-shot 프롬프트
few_shot_prompt = """다음은 클라우드 서비스에 대한 간결한 설명입니다:

서비스: Azure Virtual Machine
설명: 다양한 크기와 운영체제를 지원하는 확장 가능한 컴퓨팅 리소스로, IaaS 솔루션을 제공합니다.

서비스: AWS Lambda
설명: 서버리스 컴퓨팅 서비스로, 코드 실행만 관리하며 인프라 관리가 필요 없는 이벤트 기반 애플리케이션을 구축할 수 있습니다.

서비스: Google Cloud Storage
설명: 확장 가능하고 내구성 있는 객체 스토리지 솔루션으로, 정적 콘텐츠 호스팅부터 빅데이터 분석까지 다양한 용도로 활용됩니다.

위의 형식을 따라 아래 서비스에 대한 설명을 작성해주세요:

서비스: Azure Kubernetes Service (AKS)
설명:

서비스: Azure Functions
설명:

서비스: Azure Cosmos DB
설명:
"""
print(get_completion(few_shot_prompt))

서비스: Azure Kubernetes Service (AKS)  
설명: Kubernetes 관리의 복잡성을 줄여주는 컨테이너 오케스트레이션 서비스로, 애플리케이션을 쉽게 배포하고 관리할 수 있습니다.

서비스: Azure Functions  
설명: 서버리스 컴퓨팅 플랫폼으로, 이벤트 기반으로 코드를 실행하고 자동으로 리소스를 관리하여 개발자가 비즈니스 로직에 집중할 수 있게 합니다.

서비스: Azure Cosmos DB  
설명: 분산형 멀티모델 데이터베이스 서비스로, 글로벌 배포 및 자동 확장 기능을 제공하며, 다양한 데이터 모델과 API를 지원하여 유연한 데이터 관리를 가능하게 합니다.


## 4. 고급 프롬프트 엔지니어링 기법

이 단계에서는 LLM을 더욱 효과적으로 활용하기 위한 고급 프롬프트 엔지니어링 기법을 실습합니다.

- 시스템 메시지, 다중 단계 지시사항, 출력 형식 지정 등 실제 서비스 개발에 활용할 수 있는 고급 프롬프트 작성법을 다룹니다.
- 각 기법을 통해 LLM의 응답 품질과 일관성을 높이고, 원하는 형태의 결과를 얻는 방법을 익힐 수 있습니다.
- 실습 코드를 직접 수정·실행하며 다양한 고급 프롬프트 전략을 실험해보세요.
- 실제 프로젝트나 서비스에 적용할 수 있는 실전 노하우를 쌓는 데 도움이 됩니다.

### 4.1. 시스템 메시지 활용하기

시스템 메시지를 사용하여 LLM의 전반적인 행동을 정의할 수 있습니다.

In [9]:
def get_completion_with_system(system_message, user_message, model=deployment_name):
    """시스템 메시지와 사용자 메시지를 사용하여 Azure OpenAI API에 요청을 보내고 응답을 반환합니다."""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=800
    )
    return response.choices[0].message.content

# 기술 전문가 시스템 메시지
tech_system_message = """당신은 Microsoft Azure 전문가입니다. 항상 기술적으로 정확한 정보를 제공하며, 
복잡한 개념도 초보자가 이해할 수 있도록 쉽게 설명합니다. 가능한 경우 코드 예제나 구체적인 단계를 포함해주세요."""

user_question = "Azure Functions와 Azure Logic Apps의 차이점은 무엇인가요?"

print("=== 시스템 메시지 활용 ===\n")
print(get_completion_with_system(tech_system_message, user_question))

=== 시스템 메시지 활용 ===

Azure Functions와 Azure Logic Apps는 둘 다 서버리스 컴퓨팅을 지원하는 Azure의 서비스지만, 각각의 목적과 사용 사례가 다릅니다. 아래에서 두 서비스의 차이점을 쉽게 설명하겠습니다.

### Azure Functions
- **개념**: Azure Functions는 코드 실행에 중점을 둔 서버리스 컴퓨팅 서비스입니다. 이벤트 기반으로 작동하며, 특정 이벤트(예: HTTP 요청, 큐 메시지 등)가 발생할 때 자동으로 코드가 실행됩니다.
- **주 사용 사례**:
  - 간단한 API 구현
  - 데이터 처리
  - 자동화 작업 (예: 파일 업로드 시 처리)
- **개발 방식**: 사용자는 C#, JavaScript, Python 등 다양한 언어로 코드를 작성합니다. Azure Functions는 코드의 실행만을 관리하고, 인프라는 Azure가 자동으로 관리합니다.
- **예제**: HTTP 요청을 받아 JSON 데이터를 처리하고 결과를 반환하는 간단한 함수.
  ```csharp
  public static async Task<IActionResult> Run(
      HttpRequest req,
      ILogger log)
  {
      string requestBody = await new StreamReader(req.Body).ReadToEndAsync();
      dynamic data = JsonConvert.DeserializeObject(requestBody);
      return new OkObjectResult(data);
  }
  ```

### Azure Logic Apps
- **개념**: Azure Logic Apps는 비즈니스 프로세스를 자동화하는 데 중점을 둔 서비스입니다. 다양한 서비스와 연결하여 워크플로우를 쉽게 만들 수 있습니다.
- **주 사용 사례**:
  - 여러 시스템 간의 데이터 통합
  - 주기적인 작업 자동화 (예: 이메일 발송

### 4.2. 다중 단계 지시사항 (Multi-Step Instructions)

복잡한 작업을 여러 단계로 나누어 수행하도록 지시할 수 있습니다.

In [10]:
multi_step_prompt = """Azure Web App을 사용하여 간단한 웹 애플리케이션을 배포하는 과정을 알려주세요.
다음 순서로 설명해주세요:

1. 필요한 사전 요구 사항 나열
2. Azure Portal에서 Web App 리소스 생성 단계
3. VS Code를 사용하여 로컬에서 웹 앱 개발하기
4. Azure에 웹 앱 배포하기
5. 배포 후 모니터링 및 문제 해결 방법

각 단계마다 구체적인 명령어나 설정 옵션을 포함해주세요."""

print("=== 다중 단계 지시사항 ===\n")
print(get_completion(multi_step_prompt))

=== 다중 단계 지시사항 ===

Azure Web App을 사용하여 간단한 웹 애플리케이션을 배포하는 과정은 다음과 같습니다. 각 단계에 대한 구체적인 설명과 명령어를 포함하였습니다.

### 1. 필요한 사전 요구 사항 나열

- **Azure 계정**: Azure 포털에 로그인할 수 있는 계정이 필요합니다. 무료 계정을 생성할 수 있습니다.
- **Visual Studio Code**: 코드 편집기입니다. [공식 웹사이트](https://code.visualstudio.com/)에서 다운로드하여 설치합니다.
- **Azure CLI**: Azure 리소스를 관리하기 위한 명령줄 도구입니다. [Azure CLI 설치 가이드](https://docs.microsoft.com/cli/azure/install-azure-cli)에서 설치할 수 있습니다.
- **Node.js**: JavaScript 런타임 환경입니다. [Node.js 다운로드 페이지](https://nodejs.org/)에서 설치합니다.
- **Git**: 버전 관리 도구입니다. [Git 다운로드 페이지](https://git-scm.com/downloads)에서 설치할 수 있습니다.

### 2. Azure Portal에서 Web App 리소스 생성 단계

1. **Azure Portal 로그인**: [Azure Portal](https://portal.azure.com/)에 로그인합니다.
2. **리소스 생성**: 왼쪽 메뉴에서 "리소스 만들기"를 클릭합니다.
3. **Web App 선택**: "Web App"을 검색하여 선택합니다.
4. **리소스 설정**:
   - **구독**: 사용할 Azure 구독을 선택합니다.
   - **리소스 그룹**: 새 리소스 그룹을 만들거나 기존 리소스 그룹을 선택합니다.
   - **웹 앱 이름**: 고유한 웹 앱 이름을 입력합니다.
   - **배포 지역**: 가까운 지역을 선택합니다 (예: "한국 중부").
   - **운영 체제**: "Window

### 4.3. 출력 형식 지정 (Output Formatting)

LLM의 응답을 특정 형식으로 받을 수 있습니다.

In [11]:
format_prompt = """Azure의 주요 AI 서비스 5가지를 JSON 형식으로 제공해주세요. 각 서비스에 대해 다음 정보를 포함해주세요:
- 서비스 이름
- 간단한 설명
- 주요 기능 (3가지)
- 일반적인 사용 사례 (2가지)
- 관련 Azure 서비스 (2가지)

응답은 다음과 같은 JSON 형식이어야 합니다:
{
  "ai_services": [
    {
      "name": "서비스 이름",
      "description": "설명",
      "key_features": ["기능1", "기능2", "기능3"],
      "use_cases": ["사용 사례1", "사용 사례2"],
      "related_services": ["관련 서비스1", "관련 서비스2"]
    },
    ...
  ]
}"""

json_response = get_completion(format_prompt)
print("=== 출력 형식 지정 (JSON) ===\n")
print(json_response)

# JSON 응답을 파싱하여 구조화된 데이터로 사용 가능
try:
    parsed_response = json.loads(json_response)
    print("\n=== 첫 번째 서비스 정보 ===\n")
    print(f"서비스 이름: {parsed_response['ai_services'][0]['name']}")
    print(f"설명: {parsed_response['ai_services'][0]['description']}")
    print("주요 기능:")
    for feature in parsed_response['ai_services'][0]['key_features']:
        print(f"- {feature}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 오류: {e}")

=== 출력 형식 지정 (JSON) ===

```json
{
  "ai_services": [
    {
      "name": "Azure Cognitive Services",
      "description": "AI 서비스를 통해 이미지, 음성, 언어, 텍스트 등 다양한 데이터를 분석하고 이해할 수 있도록 지원하는 서비스 집합.",
      "key_features": ["음성 인식 및 합성", "이미지 및 비디오 분석", "자연어 처리"],
      "use_cases": ["챗봇 개발", "감정 분석"],
      "related_services": ["Azure Machine Learning", "Azure Bot Services"]
    },
    {
      "name": "Azure Machine Learning",
      "description": "ML 모델을 쉽게 구축, 학습, 배포할 수 있도록 도와주는 클라우드 기반 플랫폼.",
      "key_features": ["자동화된 ML", "모델 관리 및 배포", "다양한 프레임워크 지원"],
      "use_cases": ["예측 분석", "추천 시스템"],
      "related_services": ["Azure Databricks", "Azure Synapse Analytics"]
    },
    {
      "name": "Azure Bot Services",
      "description": "지능형 봇을 쉽게 구축하고 배포할 수 있는 플랫폼으로, 다양한 채널에 통합할 수 있음.",
      "key_features": ["다양한 채널 지원", "자연어 이해(NLU)", "봇 프레임워크 SDK"],
      "use_cases": ["고객 서비스 자동화", "상담 및 지원"],
      "related_services": ["Azure Cognitive Services", "Azure Functions"]
    },
    {
     

## 5. 실전 프롬프트 엔지니어링

이 단계에서는 실제 업무나 프로젝트에서 바로 활용할 수 있는 프롬프트 엔지니어링 예제를 다룹니다.

- 실전 시나리오(아키텍처 설계, CLI 명령어 생성 등)에 맞는 프롬프트 작성법을 실습합니다.
- LLM을 통해 복잡한 요구사항을 효과적으로 해결하는 방법을 익힐 수 있습니다.
- 다양한 실전 예제를 직접 수정·실행하며, 자신만의 프롬프트 설계 역량을 강화해보세요.
- 실제 현업에서 활용 가능한 프롬프트 작성 노하우를 쌓는 데 도움이 됩니다.

### 5.1. Azure 아키텍처 설계 조언 받기

In [12]:
architecture_prompt = """당신은 수석 Azure 솔루션 아키텍트입니다. 다음 요구사항에 맞는 Azure 아키텍처를 설계해주세요:

요구사항:
- 고가용성 웹 애플리케이션 (ASP.NET Core)
- 실시간 데이터 처리 (IoT 디바이스에서 수집된 센서 데이터)
- 대용량 데이터 저장 및 분석
- AI 기반 예측 모델을 통한 이상 징후 감지
- 철저한 보안 및 규정 준수 (개인정보 포함)
- 비용 효율적인 설계

다음 사항을 포함해주세요:
1. 아키텍처 다이어그램 (텍스트로 표현)
2. 각 Azure 서비스의 선택 이유
3. 확장성 및 장애 조치 전략
4. 예상 비용 최적화 방안
5. 보안 고려사항"""

print("=== Azure 아키텍처 설계 조언 ===\n")
print(get_completion(architecture_prompt))

=== Azure 아키텍처 설계 조언 ===

다음은 주어진 요구사항에 맞춘 Azure 아키텍처 설계입니다.

### 1. 아키텍처 다이어그램 (텍스트로 표현)

```
[IoT 디바이스] --> [IoT Hub] --> [Azure Stream Analytics] --> [Azure Functions] --> [Azure Blob Storage]
                                                        |                              |
                                                        v                              v
                                                [Azure SQL Database]         [Azure Data Lake Storage]
                                                        |
                                                        v
                                             [Azure Machine Learning]
                                                        |
                                                        v
                                            [Azure App Service (ASP.NET Core)]
                                                        |
                                                        v
                          

### 5.2. Azure CLI 명령어 생성

In [13]:
cli_prompt = """Azure CLI 명령어를 사용하여 다음 작업을 수행하는 스크립트를 생성해주세요:

1. 리소스 그룹 생성 (이름: rg-workshop-prod, 위치: Korea Central)
2. Azure Kubernetes Service(AKS) 클러스터 생성 (3개 노드, 시스템 할당 관리 ID 사용)
3. Azure Container Registry(ACR) 생성 및 AKS와 연결
4. Azure Key Vault 생성 및 필요한 비밀 저장 (DB 연결 문자열, API 키)
5. Azure Monitor를 사용하여 AKS 클러스터 모니터링 설정

각 명령어에 대한 간단한 설명을 주석으로 추가해주세요."""

print("=== Azure CLI 명령어 생성 ===\n")
print(get_completion(cli_prompt))

=== Azure CLI 명령어 생성 ===

다음은 Azure CLI를 사용하여 요구사항에 맞게 리소스를 생성하고 설정하는 스크립트입니다. 각 단계에 대한 설명을 주석으로 추가하였습니다.

```bash
#!/bin/bash

# 1. 리소스 그룹 생성
az group create --name rg-workshop-prod --location "Korea Central"
# 리소스 그룹을 생성합니다. 이름은 rg-workshop-prod이고, 위치는 Korea Central입니다.

# 2. Azure Kubernetes Service(AKS) 클러스터 생성
az aks create \
  --resource-group rg-workshop-prod \
  --name myAKSCluster \
  --node-count 3 \
  --enable-managed-identity \
  --generate-ssh-keys
# AKS 클러스터를 생성합니다. 노드 수는 3개이며, 시스템 할당 관리 ID를 사용합니다. SSH 키도 자동으로 생성됩니다.

# 3. Azure Container Registry(ACR) 생성 및 AKS와 연결
az acr create \
  --resource-group rg-workshop-prod \
  --name myContainerRegistry \
  --sku Basic
# ACR(컨테이너 레지스트리)를 생성합니다. SKU는 Basic입니다.

# ACR와 AKS를 연결하여 AKS가 ACR에서 이미지를 가져올 수 있도록 설정합니다.
az aks update \
  --resource-group rg-workshop-prod \
  --name myAKSCluster \
  --attach-acr myContainerRegistry
# AKS 클러스터가 ACR에 접근할 수 있도록 연결합니다.

# 4. Azure Key Vault 생성 및 비밀 저장
az keyvault create \
  --name myKeyVault \


## 6. 실습: 나만의 프롬프트 작성하기

이제 지금까지 배운 내용을 바탕으로 나만의 프롬프트를 작성해봅시다.

In [14]:
# 여기에 직접 프롬프트를 작성하고 실행해보세요
my_prompt = """
# 여기에 프롬프트를 작성하세요
"""

# 주석을 제거하고 실행하세요
# print(get_completion(my_prompt))

## 7. 마무리 및 다음 단계

이 세션에서는 LLM과 프롬프트 엔지니어링의 기본 및 고급 기법을 학습했습니다. 다음 세션에서는 GitHub Copilot과 AutoGen을 활용하여 Multi AI Agent 앱을 구현해볼 예정입니다.

### 학습한 내용 요약
- Azure AI Foundry에서 LLM 모델 배포 및 사용 방법
- 기본 프롬프트 엔지니어링 기법 (명확한 지시사항, 역할 부여, 단계별 사고 유도, Few-Shot 학습)
- 고급 프롬프트 엔지니어링 기법 (시스템 메시지, 다중 단계 지시사항, 출력 형식 지정)
- 실전 프롬프트 엔지니어링 사례

### 추가 학습 자료
- [Azure OpenAI Service 공식 문서](https://docs.microsoft.com/azure/cognitive-services/openai/)
- [Azure AI Foundry 공식 문서](https://docs.microsoft.com/azure/ai-services/)
- [Generative AI for Beginners (Microsoft)](https://github.com/microsoft/generative-ai-for-beginners)

### LLM 파라미터: temperature와 top-p

- **temperature**: 생성되는 텍스트의 다양성과 창의성을 조절하는 값입니다. 0에 가까울수록 더 결정적이고 일관된 답변을, 1에 가까울수록 더 다양하고 창의적인 답변을 생성합니다. 일반적으로 0.2~0.8 사이의 값을 많이 사용합니다.
- **top-p (nucleus sampling)**: 모델이 다음 단어를 선택할 때 누적 확률이 top-p 이하가 되도록 후보군을 제한합니다. 1에 가까울수록 다양한 답변이, 0.5 등 낮은 값일수록 더 보수적인 답변이 생성됩니다.
- 두 파라미터를 조합해 답변의 창의성, 다양성, 일관성을 실험할 수 있습니다.

아래 코드에서 temperature와 top_p 값을 변경해 다양한 응답을 실험해보세요.

In [ ]:
# temperature와 top_p를 조정하여 응답의 다양성 실험 (동기적으로 처리)
prompt = "Azure OpenAI의 주요 특징을 한 문장으로 설명해줘."

response1 = client.chat.completions.create(
    model=deployment_name,
    messages=[{"role": "user", "content": prompt}],
    temperature=0.1,
    top_p=0.5,
    max_tokens=50
)
print("temperature=0.2, top_p=0.5:")
print(response1.choices[0].message.content)

response2 = client.chat.completions.create(
    model=deployment_name,
    messages=[{"role": "user", "content": prompt}],
    temperature=0.9,
    top_p=1.0,
    max_tokens=50
)
print("\ntemperature=0.8, top_p=1.0:")
print(response2.choices[0].message.content)

temperature=0.2, top_p=1.0:
Azure OpenAI는 Microsoft Azure 플랫폼에서 제공되는 인공지능 모델로, 자연어 처리, 이미지 생성 등 다양한 AI 기능을 안전하고 확장 가능하게 활용할 수 있도록 지원합니다.

temperature=0.8, top_p=1.0:
Azure OpenAI는 강력한 AI 모델에 대한 안전하고 확장 가능한 접근을 제공하여 기업이 자연어 처리, 텍스트 생성, 요약 등 다양한 작업을 수행할 수 있도록 지원하는 클라우드 기반 플랫폼입니다.
